In [ ]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from tqdm import tqdm

from PIL import Image, ImageOps

In [ ]:
DATA_DIR = "/Volumes/SSD970/"

In [ ]:
bbox_df = pd.read_csv(os.path.join(DATA_DIR, 'bbox_clean.csv'))
bbox_df["UID"] = bbox_df["StudyInstanceUID"]
bbox_df = bbox_df.set_index("UID")

In [ ]:
bbox_df["StudyInstanceUID"].isnull().any()

In [ ]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
train_df["UID"] = train_df["StudyInstanceUID"]
train_df = train_df.set_index("UID")
train_df.head()

In [ ]:
pos_df = pd.read_csv(os.path.join(DATA_DIR, 'infer_sagittal_position.csv')).set_index('UID')
pos_df.head()

In [ ]:
pos_df = pos_df.reset_index().drop_duplicates(subset='UID').set_index('UID')
pos_df.head()

In [ ]:
def get_padded_position(size, pos, reshape_size=256):
    h, w = size
    pad = int((max(size) - min(size)) // 2)
    x, y = pos
    if h > w:
        x = pad + x
    else:
        y = pad + y
    scale = reshape_size / max(size)
    return int(x * scale), int(y * scale)

get_padded_position([200, 400], [200, 100])

In [ ]:
bbox_df.iloc[0]

In [ ]:
pos_df.loc[bbox_df.iloc[0].name]

In [ ]:
def get_sagittal_center_position(s):
    """
    center position w.r.t cropped label image
    이것으로써 bbox 가 sagittal image 에서 C? 을 말해주는가 알 수 있다.
    C? 이 classification label 가 되겠다.
    :param s:
    :return:
    """
    y, x = np.round(s.axial_index * s.aspect), s.coronal_index  # row, column
    UID = s["StudyInstanceUID"]
    if UID in pos_df.index:
        left, top, right, bottom = pos_df.loc[UID, ['left', 'top', 'right', 'bottom']]
    else:
        print(f"{UID} not in index")
        return
    # print([left, top, right, bottom])
    y -= left
    x -= top

    padded_pos = get_padded_position([bottom - top, right - left], [x, y])
    return padded_pos


sagittal_df = bbox_df.iloc[0]
UID = sagittal_df.name
img = Image.open(os.path.join(DATA_DIR, 'train_sagittal_labels_jpeg95', UID, f"{int(sagittal_df.sagittal_index)}.png"))
x, y = get_sagittal_center_position(sagittal_df)
_, axs = plt.subplots(1, 2)

original_img = Image.open(
    os.path.join(DATA_DIR, 'train_sagittal_images_jpeg95', UID, f"{int(sagittal_df.sagittal_index)}.jpeg"))
print(original_img.size)
axs[0].imshow(original_img, cmap='bone')
axs[0].axvline(sagittal_df.coronal_index)
axs[0].axhline(np.round(sagittal_df.axial_index * sagittal_df.aspect))

axs[1].imshow(img, cmap='nipy_spectral')
axs[1].axvline(x)
axs[1].axhline(y)

In [ ]:
bbox_df.insert(2, 'sagittal_label_x', None)
bbox_df.insert(2, 'sagittal_label_y', None)
bbox_df.head()

In [ ]:
bbox_df = bbox_df.reset_index()
bbox_df.head()

In [ ]:
bbox_df

In [ ]:
for i in tqdm(range(len(bbox_df))):
    sagittal_df = bbox_df.iloc[i]
    x, y = get_sagittal_center_position(sagittal_df)
    # label_img = Image.open(os.path.join(DATA_DIR, 'train_sagittal_labels_jpeg95', UID, f"{int(sagittal_df.sagittal_index)}.png"))
    bbox_df.loc[i, ['sagittal_label_x', 'sagittal_label_y']] = x, y
bbox_df = bbox_df.set_index('UID')
bbox_df.head()

In [ ]:
bbox_df.insert(2, 'left', None)
bbox_df.insert(2, 'top', None)
bbox_df.insert(2, 'right', None)
bbox_df.insert(2, 'bottom', None)
bbox_df.head()

In [ ]:
for UID in tqdm(bbox_df.index.values):
    bbox_df.loc[UID, ['left','top','right','bottom']] = pos_df.loc[UID, ['left','top','right','bottom']].values

In [ ]:
bbox_df.head()

In [ ]:
# bbox_df.to_csv('train_bbox_sagittal_classification.csv')